In [1]:
pip install git+https://github.com/thu-ml/zhusuan.git

In [2]:
pip install git+https://github.com/haowen-xu/tfsnippet.git@v0.1.2

In [3]:
pip install git+https://github.com/NetManAIOps/donut

In [51]:
!pip install numpy==1.18.5
!pip install tensorflow==1.15.4
import numpy as np
import pandas as pd
from donut import complete_timestamp, standardize_kpi
import time
import datetime

# Read the raw data.
df = pd.read_csv("realKnownCause/ambient_temperature_system_failure.csv")
timestamp2 = df['timestamp'].to_numpy()
values = df['value'].to_numpy()# If there is no label, simply use all zeros.
labels = np.zeros_like(values, dtype=np.int32)

timestamp = list()
for s in timestamp2:
    timestamp.append(int(time.mktime(time.strptime(s, '%Y-%m-%d %H:%M:%S'))))

# Complete the timestamp, and obtain the missing point indicators.
timestamp, missing, (values, labels) = \
    complete_timestamp(timestamp, (values, labels))

# Split the training and testing data.
test_portion = 0.3
test_n = int(len(values) * test_portion)
train_values, test_values = values[:-test_n], values[-test_n:]
train_labels, test_labels = labels[:-test_n], labels[-test_n:]
train_missing, test_missing = missing[:-test_n], missing[-test_n:]

# Standardize the training and testing data.
train_values, mean, std = standardize_kpi(
    train_values, excludes=np.logical_or(train_labels, train_missing))
test_values, _, _ = standardize_kpi(test_values, mean=mean, std=std)

In [53]:
import tensorflow as tf
from donut import Donut
from tensorflow import keras as K
from tfsnippet.modules import Sequential

# We build the entire model within the scope of `model_vs`,
# it should hold exactly all the variables of `model`, including
# the variables created by Keras layers.
with tf.variable_scope('model') as model_vs:
    model = Donut(
        h_for_p_x=Sequential([
            K.layers.Dense(100, kernel_regularizer=K.regularizers.l2(0.001),
                           activation=tf.nn.relu),
            K.layers.Dense(100, kernel_regularizer=K.regularizers.l2(0.001),
                           activation=tf.nn.relu),
        ]),
        h_for_q_z=Sequential([
            K.layers.Dense(100, kernel_regularizer=K.regularizers.l2(0.001),
                           activation=tf.nn.relu),
            K.layers.Dense(100, kernel_regularizer=K.regularizers.l2(0.001),
                           activation=tf.nn.relu),
        ]),
        x_dims=120,
        z_dims=5,
    )

In [54]:
from donut import DonutTrainer, DonutPredictor

trainer = DonutTrainer(model=model, model_vs=model_vs)
predictor = DonutPredictor(model)

with tf.Session().as_default():
    trainer.fit(train_values, train_labels, train_missing, mean, std)
    test_score = predictor.get_score(test_values, test_missing)




Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where







Trainable Parameters                     (58,150 in total)
----------------------------------------------------------
donut/p_x_given_z/x_mean/bias           (120,)         120
donut/p_x_given_z/x_mean/kernel         (100, 120)  12,000
donut/p_x_given_z/x_std/bias            (120,)         120
donut/p_x_given_z/x_std/kernel          (100, 120)  12,000
donut/q_z_given_x/z_mean/bias           (5,)             5
donut/q_z_given_x/z_mean/kernel         (100, 5)       500
donut/q_z_given_x/z_std/bias            (5,)             5
donut/q_z_given_x/z_std/kernel          (100, 5)       500
sequential/forward/_0/dense/bias        (100,)         100
sequential/forward/_0/dense/kerne

[Epoch 115/256, Step 1600, ETA 21.66s] step time: 0.00908s (±0.02975s); valid time: 0.2969s; loss: 71.3056 (±2.52188); valid loss: 145.326 (*)
[Epoch 120/256, Step 1680, ETA 20.41s] Learning rate decreased to 3.167635202407837e-05
[Epoch 122/256, Step 1700, ETA 20.14s] step time: 0.00606s (±0.006877s); valid time: 0s; loss: 70.9033 (±2.87624); valid loss: 147.709
[Epoch 129/256, Step 1800, ETA 18.75s] step time: 0.005772s (±0.00493s); valid time: 0s; loss: 70.1501 (±2.78843); valid loss: 153.837
[Epoch 130/256, Step 1820, ETA 18.57s] Learning rate decreased to 2.3757264018058778e-05
[Epoch 136/256, Step 1900, ETA 17.7s] step time: 0.008769s (±0.007003s); valid time: 0.01599s; loss: 69.3398 (±2.80185); valid loss: 149.946
[Epoch 140/256, Step 1960, ETA 17.25s] Learning rate decreased to 1.7817948013544083e-05
[Epoch 143/256, Step 2000, ETA 16.75s] step time: 0.01s (±0.005225s); valid time: 0.01563s; loss: 70.019 (±2.57234); valid loss: 148.816
[Epoch 150/256, Step 2100, ETA 15.46s] step

In [63]:
print(min(test_score), max(test_score))

-591.5387 -0.3574451
